In [25]:
%load_ext autoreload
%autoreload 2
%connect_info
%pprint 1
# General imports
import numpy as np
import pandas as pd
import logging
import matplotlib.pyplot as plt
from typing import NamedTuple, List,Tuple
from dataclasses import dataclass
from gt.solutions.br import BrownRobinsonOptimizer
from IPython.display import display

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')
logger = logging.getLogger(__name__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{
  "shell_port": 64464,
  "iopub_port": 64465,
  "stdin_port": 64466,
  "control_port": 64467,
  "hb_port": 64468,
  "ip": "127.0.0.1",
  "key": "b0a6862a-e3ad4974ce03be4b267392ad",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-7e85f7bb-e36e-425c-a524-a3461bbb837d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
Pretty printing has been turned OFF


In [21]:
#

original_matrix = np.array([
    [-5, -1, -1, -3, 0],
    [-4, 1, 0, -4, 0], 
    [-6, -1, 0, -5, 0], 
    [-3, 0, 0, -3, 0], 
    [2, -3, -4, 3, -3]
])

def normalize(m: np.ndarray) -> Tuple[np.ndarray, int]:
    norm_element = -np.min(m)
    if norm_element < 0:
        return m, 0
    return m + norm_element, norm_element


normalized_matrix, norm_element = normalize(original_matrix)
normalized_matrix

opt = BrownRobinsonOptimizer(original_matrix, max_steps=10000)
(A, B), states = opt.fit(0.001)
states[-1]

OptimizerState(step=2136, game_cost_current=Fraction(1, 1068), A_income=array([-5748., -3612., -5418., -2709., -3126.]), B_loss=array([-2850., -2137., -2849., -2136., -2136.]), A_frequencies=array([[1.000e+00, 0.000e+00, 0.000e+00, 1.423e+03, 7.120e+02]]), B_frequencies=array([[ 903.,    0., 1233.,    0.,    0.]]), A_strategy=3, B_strategy=2, min_win=Fraction(-4, 3), max_loss=Fraction(-475, 356))

In [ ]:
def to_df(matrix: np.ndarray) -> pd.DataFrame:
    return pd.DataFrame(data=matrix,
                        index=[f'a{i+1}' for i in range(matrix.shape[0])],
                        columns=[f'b{i+1}' for i in range(matrix.shape[1])])
df = to_df(normalized_matrix)

In [47]:
def transform_matrix(matrix: pd.DataFrame) -> pd.DataFrame:
    # eliminate strictly dominated strategies
    columns = matrix.columns
    filtered_columns = []
    for column in columns:
        logger.debug(f"Comparing column {column}")
        for compare_column in columns:
            if column != compare_column and (matrix[compare_column] >= matrix[column]).all():
                logger.info(f"Column {column} ({matrix[column].ravel()}) "
                            f"is strictly dominated by {compare_column} ({matrix[compare_column].ravel()})")
                break
        else:
            filtered_columns.append(column)
    
    rows = matrix.index
    filtered_rows = []
    for row in rows:
        logger.debug(f"Comparing row {row}")
        for compare_row in rows:
            if row != compare_row and (matrix.loc[compare_row] >= matrix.loc[row]).all():
                logger.info(f"Row {row} ({matrix.loc[row].ravel()}) is "
                            f"strictly dominated by {compare_row} ({matrix.loc[compare_row].ravel()})")
                break
        else:
            filtered_rows.append(row)
    return matrix[filtered_columns].loc[filtered_rows]

transform_matrix(df)
#(df.loc['a1'] >= df.loc['a2']).all()

03:11:12 DEBUG:Comparing column b1
03:11:12 INFO:Column b1 ([1 2 0 3 8]) is strictly dominated by b4 ([3 2 1 3 9])
03:11:12 DEBUG:Comparing column b2
03:11:12 DEBUG:Comparing column b3
03:11:12 INFO:Column b3 ([5 6 6 6 2]) is strictly dominated by b5 ([6 6 6 6 3])
03:11:12 DEBUG:Comparing column b4
03:11:12 DEBUG:Comparing column b5
03:11:12 DEBUG:Comparing row a1
03:11:12 INFO:Row a1 ([1 5 5 3 6]) is strictly dominated by a4 ([3 6 6 3 6])
03:11:12 DEBUG:Comparing row a2
03:11:12 DEBUG:Comparing row a3
03:11:12 INFO:Row a3 ([0 5 6 1 6]) is strictly dominated by a2 ([2 7 6 2 6])
03:11:12 DEBUG:Comparing row a4
03:11:12 DEBUG:Comparing row a5


,b2,b4,b5
a2,7,2,6
a4,6,3,6
a5,3,9,3
